In [1]:
import cv2
import numpy as np
from mtcnn import MTCNN
import joblib
import torch
from facenet_pytorch import InceptionResnetV1
import torchvision.transforms as transforms
import os
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
knn_ex_model = joblib.load(r"D:\GitHub\Machine_Learning\weights\knn_model_ex_optimal.joblib")
labels = ['Thang', 'Su', 'Nhung', 'Tuyen', 'Vu', 'Dat', 'Huy']
feature_extractor = InceptionResnetV1(pretrained='vggface2', classify=False).eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
similarity_threshold = 0.8
detector = MTCNN()
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    image = frame.copy()
    faces = detector.detect_faces(image)

    if faces:
        for face in faces:
            x, y, w, h = face['box']
            face_crop = image[y:y+h, x:x+w]
            if face_crop.shape[0] == 0 or face_crop.shape[1] == 0:
                continue

            face_resized = cv2.resize(face_crop, (160, 160))
            face_tensor = transform(face_resized).unsqueeze(0)

            with torch.no_grad():
                face_features = feature_extractor(face_tensor)

            face_features = face_features.cpu().numpy().flatten().reshape(1, -1)
            knn_label_idx = knn_ex_model.predict(face_features)[0]
            knn_label = labels[knn_label_idx]

            distances = cosine_similarity(face_features, knn_ex_model._fit_X)
            if np.max(distances) < similarity_threshold:
                knn_label = "Unknown"

            print(f'Predicted label: {knn_label}')
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f'KNN: {knn_label}', (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    else:
        cv2.putText(image, 'No face detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Face Recognition', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
